#Librerías


In [ ]:
#Uso de librerías. Celda nº1
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import warnings
from collections import defaultdict
from tabulate import tabulate
from google.colab import data_table
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Preprocesamiento

In [ ]:
# Cargar el archivo Excel
xls = pd.ExcelFile("Test_animales.xlsx")
# Leer las hojas y asignar la nueva variable "degenera"
df_list = []
for i, sheet in enumerate(xls.sheet_names):
    df = xls.parse(sheet)
    df["degenera"] = i  # Asigna 0 a la primera hoja, 1 a la segunda, etc.
    df_list.append(df)

# Unir todas las hojas en un solo DataFrame
df = pd.concat(df_list, ignore_index=True)
# Guardar el resultado en un CSV
df.to_csv("Test_animales.csv", index=False)


data = pd.read_csv("Test_animales.csv", header =2)
# Asignar la primera fila como nombres de columnas
data.columns = data.iloc[0]  # Toma la primera fila como encabezados
data = data[1:].reset_index(drop=True)  # Eliminar la fila duplicada y resetear índice

# Asegurar que no hay nombres de columna repetidos
data.columns = [str(col) for col in data.columns]
display(data)

,Edad sacrificio,ID animal,Sexo,P30,3m,4m,6m,9m,12m,17-21m,...,OI,OD,OI,F.Optomotor,Edad Opto,Dato OPT,F.OCT,OD (400 um),OI,0
0,1,WT-25,♀,110220-5,NaN,NaN,NaN,NaN,NaN,NaN,...,19.5,146.7,166.1,2020-02-17 00:00:00,"1,3m",0.572,2020-02-17 00:00:00,52.818,202.818,0
1,1,WT-110,♀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,WT-111,♂,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,WT-112,♀,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,WT-113,♂,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,P11,R10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
255,P11,R11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
256,P11,R12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
257,P11,R13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# Contador para cada nombre de columna duplicada
oi_counter = 1
od_counter = 1
new_columns = []

# Renombrar correctamente las columnas duplicadas de OI y OD
for col in data.columns:
    if col.startswith("OI"):
        new_columns.append(f"OI_{oi_counter}")
        oi_counter += 1
    elif col.startswith("OD"):
        new_columns.append(f"OD_{od_counter}")
        od_counter += 1
    elif col.startswith("0"):
        new_columns.append("degenera")
    else:
        new_columns.append(col) # Mantener los demás nombres iguales
# Aplicar los nuevos nombres
data.columns = new_columns
# Mostrar el resultado
print(data.columns)


Index(['Edad sacrificio', 'ID animal', 'Sexo', 'P30', '3m', '4m', '6m', '9m',
       '12m', '17-21m', 'OD_1', 'OI_1', 'OD_2', 'OI_2', 'OD_3', 'OI_3', 'OD_4',
       'OI_4', 'F.Optomotor', 'Edad Opto', 'Dato OPT', 'F.OCT', 'OD_5', 'OI_5',
       'degenera'],
      dtype='object')


In [ ]:
# Contar las instancias de cada valor en "degenera"
conteo = data["degenera"].value_counts()
print(conteo)

degenera
0    131
1    128
Name: count, dtype: int64


In [ ]:
# Contar valores NaN en cada columna
nan_counts = data.isna().sum()
print(tabulate(nan_counts.reset_index(), headers=["Variable", "Cantidad de NaN"], tablefmt="pretty"))

+----+-----------------+-----------------+
|    |    Variable     | Cantidad de NaN |
+----+-----------------+-----------------+
| 0  | Edad sacrificio |        4        |
| 1  |    ID animal    |        1        |
| 2  |      Sexo       |       15        |
| 3  |       P30       |       246       |
| 4  |       3m        |       236       |
| 5  |       4m        |       252       |
| 6  |       6m        |       243       |
| 7  |       9m        |       243       |
| 8  |       12m       |       239       |
| 9  |     17-21m      |       241       |
| 10 |      OD_1       |       180       |
| 11 |      OI_1       |       184       |
| 12 |      OD_2       |       181       |
| 13 |      OI_2       |       184       |
| 14 |      OD_3       |       180       |
| 15 |      OI_3       |       184       |
| 16 |      OD_4       |       181       |
| 17 |      OI_4       |       184       |
| 18 |   F.Optomotor   |       160       |
| 19 |    Edad Opto    |       154       |
| 20 |    D

In [ ]:
# Seleccionar columnas que empiezan con "OI", "OD" o "Dato OPT"
oi_od_columns = [col for col in data.columns if col.startswith("OI") or col.startswith("OD") or col.startswith("Dato OPT")]
# Eliminar filas donde TODAS esas columnas sean NaN (si al menos una tiene valor, la fila se queda)
data = data.dropna(subset=oi_od_columns, how="all").reset_index(drop=True)
data.to_csv("Test_animales.csv", index=False)
display(data)

,Edad sacrificio,ID animal,Sexo,P30,3m,4m,6m,9m,12m,17-21m,...,OI_3,OD_4,OI_4,F.Optomotor,Edad Opto,Dato OPT,F.OCT,OD_5,OI_5,degenera
0,1,WT-25,♀,110220-5,NaN,NaN,NaN,NaN,NaN,NaN,...,19.5,146.7,166.1,2020-02-17 00:00:00,"1,3m",0.572,2020-02-17 00:00:00,52.818,202.818,0
1,1.3,WT-23,♀,110220-3,NaN,NaN,NaN,NaN,NaN,NaN,...,19.9,123.1,135.2,2020-02-17 00:00:00,"1,3m",0.574,2020-02-17 00:00:00,47.365,202.364,0
2,1.3,WT-24,♀,110220-4,NaN,NaN,NaN,NaN,NaN,NaN,...,19.3,184.3,183.7,2020-02-17 00:00:00,"1,3m",0.594,2020-02-17 00:00:00,54.94,209.455,0
3,1.3,WT-26,♂,110220-1,NaN,NaN,NaN,NaN,NaN,NaN,...,18.0,131.8,166.9,2020-02-17 00:00:00,"1,3m",0.550,2020-02-17 00:00:00,57.273,212.909,0
4,1.3,WT-27,♂,110220-2,NaN,NaN,NaN,NaN,NaN,NaN,...,16.5,181.2,142.6,2020-02-17 00:00:00,"1,3m",0.616,2020-02-17 00:00:00,51.091,202.545,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,19,12120,♀,NaN,NaN,NaN,NaN,NaN,NaN,110521-5,...,3,52,"84,8",NaN,NaN,"0,308",2021-04-28 00:00:00,NaN,NaN,1
103,20,12004,♀,NaN,NaN,NaN,NaN,NaN,NaN,190521-2,...,"3,2","63,1","57,1",NaN,NaN,"0,297",2021-04-26 00:00:00,NaN,NaN,1
104,20,12045,♂,NaN,NaN,NaN,NaN,NaN,NaN,190521-7,...,4,"93,2",85,NaN,NaN,"0,319",2021-04-27 00:00:00,NaN,NaN,1
105,20,12046,♂,NaN,NaN,NaN,NaN,NaN,NaN,190521-6,...,7,"52,5","91,7",NaN,NaN,"0,308",2021-04-27 00:00:00,NaN,NaN,1


In [ ]:
data.drop('Edad sacrificio',axis =1, inplace =True)
data.drop('F.Optomotor',axis =1, inplace =True)
data.drop('F.OCT',axis =1, inplace =True)
data.drop('ID animal',axis =1, inplace =True)
display(data)

,Sexo,P30,3m,4m,6m,9m,12m,17-21m,OD_1,OI_1,...,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera
0,♀,110220-5,NaN,NaN,NaN,NaN,NaN,NaN,224.1,262.1,...,628.0,16.8,19.5,146.7,166.1,"1,3m",0.572,52.818,202.818,0
1,♀,110220-3,NaN,NaN,NaN,NaN,NaN,NaN,147.6,217.6,...,501.6,21.6,19.9,123.1,135.2,"1,3m",0.574,47.365,202.364,0
2,♀,110220-4,NaN,NaN,NaN,NaN,NaN,NaN,318.8,291.6,...,630.5,14.2,19.3,184.3,183.7,"1,3m",0.594,54.94,209.455,0
3,♂,110220-1,NaN,NaN,NaN,NaN,NaN,NaN,206.7,197.6,...,492.3,14.3,18.0,131.8,166.9,"1,3m",0.550,57.273,212.909,0
4,♂,110220-2,NaN,NaN,NaN,NaN,NaN,NaN,292.1,198.6,...,470.4,17.5,16.5,181.2,142.6,"1,3m",0.616,51.091,202.545,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,♀,NaN,NaN,NaN,NaN,NaN,NaN,110521-5,"78,1","72,1",...,"181,2",2,3,52,"84,8",NaN,"0,308",NaN,NaN,1
103,♀,NaN,NaN,NaN,NaN,NaN,NaN,190521-2,"65,1","60,8",...,"188,7",5,"3,2","63,1","57,1",NaN,"0,297",NaN,NaN,1
104,♂,NaN,NaN,NaN,NaN,NaN,NaN,190521-7,"92,2","83,8",...,"246,8",3,4,"93,2",85,NaN,"0,319",NaN,NaN,1
105,♂,NaN,NaN,NaN,NaN,NaN,NaN,190521-6,79,"113,1",...,"249,4",5,7,"52,5","91,7",NaN,"0,308",NaN,NaN,1


In [ ]:
data["Sexo"] = data["Sexo"].map({"♀": 0, "♂": 1})
data["Sexo"] = data["Sexo"].fillna(-1).astype(int)
print(data["Sexo"].value_counts())
display(data)

Sexo
 1    56
 0    44
-1     7
Name: count, dtype: int64


,Sexo,P30,3m,4m,6m,9m,12m,17-21m,OD_1,OI_1,...,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera
0,0,110220-5,NaN,NaN,NaN,NaN,NaN,NaN,224.1,262.1,...,628.0,16.8,19.5,146.7,166.1,"1,3m",0.572,52.818,202.818,0
1,0,110220-3,NaN,NaN,NaN,NaN,NaN,NaN,147.6,217.6,...,501.6,21.6,19.9,123.1,135.2,"1,3m",0.574,47.365,202.364,0
2,0,110220-4,NaN,NaN,NaN,NaN,NaN,NaN,318.8,291.6,...,630.5,14.2,19.3,184.3,183.7,"1,3m",0.594,54.94,209.455,0
3,1,110220-1,NaN,NaN,NaN,NaN,NaN,NaN,206.7,197.6,...,492.3,14.3,18.0,131.8,166.9,"1,3m",0.550,57.273,212.909,0
4,1,110220-2,NaN,NaN,NaN,NaN,NaN,NaN,292.1,198.6,...,470.4,17.5,16.5,181.2,142.6,"1,3m",0.616,51.091,202.545,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,0,NaN,NaN,NaN,NaN,NaN,NaN,110521-5,"78,1","72,1",...,"181,2",2,3,52,"84,8",NaN,"0,308",NaN,NaN,1
103,0,NaN,NaN,NaN,NaN,NaN,NaN,190521-2,"65,1","60,8",...,"188,7",5,"3,2","63,1","57,1",NaN,"0,297",NaN,NaN,1
104,1,NaN,NaN,NaN,NaN,NaN,NaN,190521-7,"92,2","83,8",...,"246,8",3,4,"93,2",85,NaN,"0,319",NaN,NaN,1
105,1,NaN,NaN,NaN,NaN,NaN,NaN,190521-6,79,"113,1",...,"249,4",5,7,"52,5","91,7",NaN,"0,308",NaN,NaN,1


In [ ]:
data["Edad_ERG"] = np.nan
edad_map = {
    "P30": 1,
    "3m": 3,
    "4m": 4,
    "6m": 6,
    "9m": 9,
    "12m": 12,
    "17-21m": 19
}

for col, val in edad_map.items():
    data.loc[data[col].notna(), "Edad_ERG"] = val

# Normalizar con Min-Max Scaling
min_val = min(edad_map.values())  # Mínimo (1)
max_val = max(edad_map.values())  # Máximo (19)
data["Edad_ERG"] = (data["Edad_ERG"] - min_val) / (max_val - min_val)
# Crear diccionario con valores normalizados
edad_normalizada = {
    nombre: round((valor - min_val) / (max_val - min_val), 3)
    for nombre, valor in edad_map.items()
}

# Mostrar
for nombre, valor_norm in edad_normalizada.items():
    print(f"{nombre}: {valor_norm}")

display(data[["P30", "3m", "4m", "6m", "9m", "12m", "17-21m", "Edad_ERG"]])

P30: 0.0
3m: 0.111
4m: 0.167
6m: 0.278
9m: 0.444
12m: 0.611
17-21m: 1.0


,P30,3m,4m,6m,9m,12m,17-21m,Edad_ERG
0,110220-5,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,110220-3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,110220-4,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,110220-1,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,110220-2,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
102,NaN,NaN,NaN,NaN,NaN,NaN,110521-5,1.0
103,NaN,NaN,NaN,NaN,NaN,NaN,190521-2,1.0
104,NaN,NaN,NaN,NaN,NaN,NaN,190521-7,1.0
105,NaN,NaN,NaN,NaN,NaN,NaN,190521-6,1.0


In [ ]:
data = data.drop(columns=["P30", "3m", "4m", "6m", "9m", "12m", "17-21m"])
display(data)

,Sexo,OD_1,OI_1,OD_2,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera,Edad_ERG
0,0,224.1,262.1,540.4,628.0,16.8,19.5,146.7,166.1,"1,3m",0.572,52.818,202.818,0,0.0
1,0,147.6,217.6,320.6,501.6,21.6,19.9,123.1,135.2,"1,3m",0.574,47.365,202.364,0,0.0
2,0,318.8,291.6,719.6,630.5,14.2,19.3,184.3,183.7,"1,3m",0.594,54.94,209.455,0,0.0
3,1,206.7,197.6,527.3,492.3,14.3,18.0,131.8,166.9,"1,3m",0.550,57.273,212.909,0,0.0
4,1,292.1,198.6,696.6,470.4,17.5,16.5,181.2,142.6,"1,3m",0.616,51.091,202.545,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,0,"78,1","72,1","174,8","181,2",2,3,52,"84,8",NaN,"0,308",NaN,NaN,1,1.0
103,0,"65,1","60,8","199,9","188,7",5,"3,2","63,1","57,1",NaN,"0,297",NaN,NaN,1,1.0
104,1,"92,2","83,8","264,3","246,8",3,4,"93,2",85,NaN,"0,319",NaN,NaN,1,1.0
105,1,79,"113,1","191,6","249,4",5,7,"52,5","91,7",NaN,"0,308",NaN,NaN,1,1.0


In [ ]:
data = data.drop_duplicates()
data = data.reset_index(drop=True)
display(data)

,Sexo,OD_1,OI_1,OD_2,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera,Edad_ERG
0,0,224.1,262.1,540.4,628.0,16.8,19.5,146.7,166.1,"1,3m",0.572,52.818,202.818,0,0.0
1,0,147.6,217.6,320.6,501.6,21.6,19.9,123.1,135.2,"1,3m",0.574,47.365,202.364,0,0.0
2,0,318.8,291.6,719.6,630.5,14.2,19.3,184.3,183.7,"1,3m",0.594,54.94,209.455,0,0.0
3,1,206.7,197.6,527.3,492.3,14.3,18.0,131.8,166.9,"1,3m",0.550,57.273,212.909,0,0.0
4,1,292.1,198.6,696.6,470.4,17.5,16.5,181.2,142.6,"1,3m",0.616,51.091,202.545,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,0,"78,1","72,1","174,8","181,2",2,3,52,"84,8",NaN,"0,308",NaN,NaN,1,1.0
103,0,"65,1","60,8","199,9","188,7",5,"3,2","63,1","57,1",NaN,"0,297",NaN,NaN,1,1.0
104,1,"92,2","83,8","264,3","246,8",3,4,"93,2",85,NaN,"0,319",NaN,NaN,1,1.0
105,1,79,"113,1","191,6","249,4",5,7,"52,5","91,7",NaN,"0,308",NaN,NaN,1,1.0


In [ ]:
data = data.drop(index=[47,48, 49])
data = data.reset_index(drop=True)
data = data.replace({'\*': ''}, regex=True)
data = data.replace({'m': '', 'M': ''}, regex=True)
data = data.replace({',': '.'}, regex=True)
display(data)

,Sexo,OD_1,OI_1,OD_2,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera,Edad_ERG
0,0,224.1,262.1,540.4,628.0,16.8,19.5,146.7,166.1,1.3,0.572,52.818,202.818,0,0.0
1,0,147.6,217.6,320.6,501.6,21.6,19.9,123.1,135.2,1.3,0.574,47.365,202.364,0,0.0
2,0,318.8,291.6,719.6,630.5,14.2,19.3,184.3,183.7,1.3,0.594,54.94,209.455,0,0.0
3,1,206.7,197.6,527.3,492.3,14.3,18.0,131.8,166.9,1.3,0.550,57.273,212.909,0,0.0
4,1,292.1,198.6,696.6,470.4,17.5,16.5,181.2,142.6,1.3,0.616,51.091,202.545,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0,78.1,72.1,174.8,181.2,2,3,52,84.8,NaN,0.308,NaN,NaN,1,1.0
100,0,65.1,60.8,199.9,188.7,5,3.2,63.1,57.1,NaN,0.297,NaN,NaN,1,1.0
101,1,92.2,83.8,264.3,246.8,3,4,93.2,85,NaN,0.319,NaN,NaN,1,1.0
102,1,79,113.1,191.6,249.4,5,7,52.5,91.7,NaN,0.308,NaN,NaN,1,1.0


In [ ]:

display(data)

,Sexo,OD_1,OI_1,OD_2,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera,Edad_ERG
0,0,224.1,262.1,540.4,628.0,16.8,19.5,146.7,166.1,1.3,0.572,52.818,202.818,0,0.0
1,0,147.6,217.6,320.6,501.6,21.6,19.9,123.1,135.2,1.3,0.574,47.365,202.364,0,0.0
2,0,318.8,291.6,719.6,630.5,14.2,19.3,184.3,183.7,1.3,0.594,54.94,209.455,0,0.0
3,1,206.7,197.6,527.3,492.3,14.3,18.0,131.8,166.9,1.3,0.550,57.273,212.909,0,0.0
4,1,292.1,198.6,696.6,470.4,17.5,16.5,181.2,142.6,1.3,0.616,51.091,202.545,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0,78.1,72.1,174.8,181.2,2,3,52,84.8,NaN,0.308,NaN,NaN,1,1.0
100,0,65.1,60.8,199.9,188.7,5,3.2,63.1,57.1,NaN,0.297,NaN,NaN,1,1.0
101,1,92.2,83.8,264.3,246.8,3,4,93.2,85,NaN,0.319,NaN,NaN,1,1.0
102,1,79,113.1,191.6,249.4,5,7,52.5,91.7,NaN,0.308,NaN,NaN,1,1.0


In [ ]:
# Contar las instancias de cada valor en "degenera"
conteo = data["degenera"].value_counts()
print(conteo)

degenera
1    57
0    47
Name: count, dtype: int64


In [ ]:
# Reemplazar "-" y "?" por NaN
data.replace({"-": np.nan, "?": np.nan}, inplace=True)
# Lista de columnas a verificar
columnas_verificar = ["OD_1", "OI_1", "OD_2", "OI_2", "OD_3", "OI_3", "OD_4", "OI_4","OD_5","OI_5","Edad Opto","Dato OPT"]
# Eliminar filas donde TODAS estas columnas sean NaN
data = data.dropna(subset=columnas_verificar, how='all')

# Convert only those
data = data.apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)
# Verificar que todo es int
print(data.dtypes)
display(data)
data.to_csv("Test_animales.csv", index=False)

Sexo         float64
OD_1         float64
OI_1         float64
OD_2         float64
OI_2         float64
OD_3         float64
OI_3         float64
OD_4         float64
OI_4         float64
Edad Opto    float64
Dato OPT     float64
OD_5         float64
OI_5         float64
degenera     float64
Edad_ERG     float64
dtype: object


,Sexo,OD_1,OI_1,OD_2,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera,Edad_ERG
0,0.0,224.1,262.1,540.4,628.0,16.8,19.5,146.7,166.1,1.3,0.572,52.818,202.818,0.0,0.0
1,0.0,147.6,217.6,320.6,501.6,21.6,19.9,123.1,135.2,1.3,0.574,47.365,202.364,0.0,0.0
2,0.0,318.8,291.6,719.6,630.5,14.2,19.3,184.3,183.7,1.3,0.594,54.940,209.455,0.0,0.0
3,1.0,206.7,197.6,527.3,492.3,14.3,18.0,131.8,166.9,1.3,0.550,57.273,212.909,0.0,0.0
4,1.0,292.1,198.6,696.6,470.4,17.5,16.5,181.2,142.6,1.3,0.616,51.091,202.545,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.0,78.1,72.1,174.8,181.2,2.0,3.0,52.0,84.8,0.0,0.308,0.000,0.000,1.0,1.0
100,0.0,65.1,60.8,199.9,188.7,5.0,3.2,63.1,57.1,0.0,0.297,0.000,0.000,1.0,1.0
101,1.0,92.2,83.8,264.3,246.8,3.0,4.0,93.2,85.0,0.0,0.319,0.000,0.000,1.0,1.0
102,1.0,79.0,113.1,191.6,249.4,5.0,7.0,52.5,91.7,0.0,0.308,0.000,0.000,1.0,1.0


In [ ]:
# Definir características (X) y variable objetivo (y)
X = data.drop(columns=['degenera'])  # Eliminar la columna objetivo
y = data['degenera']  # Variable objetivo
# Dividir en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Escalar los datos (solo para modelos que lo necesitan)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print( X_train_scaled)
display (data)

[[-0.77990776 -1.1594742  -1.19428214 ...  0.97345399  0.8173628
  -0.9169251 ]
 [-0.77990776  0.43941929  0.64115237 ...  0.98278454  0.77538424
   0.42118098]
 [-2.48339049 -1.1594742  -1.19428214 ...  0.25030397  0.50791938
  -0.9169251 ]
 ...
 [ 0.92357497 -0.31143279  0.19240886 ...  0.06654985  0.34977229
   0.92297076]
 [ 0.92357497  1.04663759  0.9924229  ...  0.70575576  0.71193857
  -0.9169251 ]
 [ 0.92357497 -0.48000897 -0.15426389 ... -1.48373544 -1.51763681
   2.09381357]]


,Sexo,OD_1,OI_1,OD_2,OI_2,OD_3,OI_3,OD_4,OI_4,Edad Opto,Dato OPT,OD_5,OI_5,degenera,Edad_ERG
0,0.0,224.1,262.1,540.4,628.0,16.8,19.5,146.7,166.1,1.3,0.572,52.818,202.818,0.0,0.0
1,0.0,147.6,217.6,320.6,501.6,21.6,19.9,123.1,135.2,1.3,0.574,47.365,202.364,0.0,0.0
2,0.0,318.8,291.6,719.6,630.5,14.2,19.3,184.3,183.7,1.3,0.594,54.940,209.455,0.0,0.0
3,1.0,206.7,197.6,527.3,492.3,14.3,18.0,131.8,166.9,1.3,0.550,57.273,212.909,0.0,0.0
4,1.0,292.1,198.6,696.6,470.4,17.5,16.5,181.2,142.6,1.3,0.616,51.091,202.545,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.0,78.1,72.1,174.8,181.2,2.0,3.0,52.0,84.8,0.0,0.308,0.000,0.000,1.0,1.0
100,0.0,65.1,60.8,199.9,188.7,5.0,3.2,63.1,57.1,0.0,0.297,0.000,0.000,1.0,1.0
101,1.0,92.2,83.8,264.3,246.8,3.0,4.0,93.2,85.0,0.0,0.319,0.000,0.000,1.0,1.0
102,1.0,79.0,113.1,191.6,249.4,5.0,7.0,52.5,91.7,0.0,0.308,0.000,0.000,1.0,1.0


#Arquitecturas Machine Learning

## Hiperparámetros por defecto

In [ ]:
# Entrenar modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 🔹 Accuracy en entrenamiento
y_pred_train = rf.predict(X_train)
train_accuracy_rf = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test (validación)
y_pred_rf = rf.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Mostrar resultados
print(f"Modelo: Random Forest")
print(f"Accuracy en entrenamiento: {train_accuracy_rf:.4f}")
print(f"Accuracy en test: {test_accuracy_rf:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_rf)}")
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(f"Matriz de Confusión para Random Forest:\n", cm_rf)

Modelo: Random Forest
Accuracy en entrenamiento: 1.0000
Accuracy en test: 1.0000

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        11

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21

Matriz de Confusión para Random Forest:
 [[10  0]
 [ 0 11]]


In [ ]:
# Escalar las características para el modelo SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train_scaled, y_train)

# 🔹 Accuracy en entrenamiento
y_pred_train = svm.predict(X_train_scaled)
train_accuracy_svm = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test
y_pred_test = svm.predict(X_test_scaled)
test_accuracy_svm = accuracy_score(y_test, y_pred_test)

# Mostrar resultados
print(f"Modelo: SVM")
print(f"Accuracy en entrenamiento: {train_accuracy_svm:.4f}")
print(f"Accuracy en test: {test_accuracy_svm:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_test)}")

Modelo: SVM
Accuracy en entrenamiento: 0.7711
Accuracy en test: 0.8095

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75        10
         1.0       0.73      1.00      0.85        11

    accuracy                           0.81        21
   macro avg       0.87      0.80      0.80        21
weighted avg       0.86      0.81      0.80        21



In [ ]:
# Importar librerías necesarias
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# XGBoost
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train)

# 🔹 Accuracy en entrenamiento
y_pred_train = xgb.predict(X_train)
train_accuracy_xgb = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test
y_pred_test = xgb.predict(X_test)
test_accuracy_xgb = accuracy_score(y_test, y_pred_test)

# Mostrar resultados
print(f"Modelo: XGBoost")
print(f"Accuracy en entrenamiento: {train_accuracy_xgb:.4f}")
print(f"Accuracy en test: {test_accuracy_xgb:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_test)}")

Modelo: XGBoost
Accuracy en entrenamiento: 1.0000
Accuracy en test: 0.9048

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      0.80      0.89        10
         1.0       0.85      1.00      0.92        11

    accuracy                           0.90        21
   macro avg       0.92      0.90      0.90        21
weighted avg       0.92      0.90      0.90        21



##GridSearchCV

In [ ]:
# Entrenar modelo Random Forest
rf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
rf.fit(X_train, y_train)

# 🔹 Accuracy en entrenamiento
y_pred_train = rf.predict(X_train)
train_accuracy_rf = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test (validación)
y_pred_rf = rf.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Mostrar resultados
print(f"Modelo: Random Forest")
print(f"Accuracy en entrenamiento: {train_accuracy_rf:.4f}")
print(f"Accuracy en test: {test_accuracy_rf:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_rf)}")
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(f"Matriz de Confusión para Random Forest:\n", cm_rf)

Modelo: Random Forest
Accuracy en entrenamiento: 1.0000
Accuracy en test: 0.8095

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75        10
         1.0       0.73      1.00      0.85        11

    accuracy                           0.81        21
   macro avg       0.87      0.80      0.80        21
weighted avg       0.86      0.81      0.80        21

Matriz de Confusión para Random Forest:
 [[ 6  4]
 [ 0 11]]


In [ ]:
# Escalar las características para el modelo SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM
svm = SVC(C=1, kernel='rbf', gamma="scale", probability=True, random_state=42)
svm.fit(X_train_scaled, y_train)

# 🔹 Accuracy en entrenamiento
y_pred_train = svm.predict(X_train_scaled)
train_accuracy_svm = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test
y_pred_test = svm.predict(X_test_scaled)
test_accuracy_svm = accuracy_score(y_test, y_pred_test)

# Mostrar resultados
print(f"Modelo: SVM")
print(f"Accuracy en entrenamiento: {train_accuracy_svm:.4f}")
print(f"Accuracy en test: {test_accuracy_svm:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_test)}")

Modelo: SVM
Accuracy en entrenamiento: 0.7711
Accuracy en test: 0.8095

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75        10
         1.0       0.73      1.00      0.85        11

    accuracy                           0.81        21
   macro avg       0.87      0.80      0.80        21
weighted avg       0.86      0.81      0.80        21



In [ ]:
# Importar librerías necesarias
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# XGBoost
xgb = XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.1, random_state=42)
xgb.fit(X_train, y_train)

# 🔹 Accuracy en entrenamiento
y_pred_train = xgb.predict(X_train)
train_accuracy_xgb = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test
y_pred_test = xgb.predict(X_test)
test_accuracy_xgb = accuracy_score(y_test, y_pred_test)

# Mostrar resultados
print(f"Modelo: XGBoost")
print(f"Accuracy en entrenamiento: {train_accuracy_xgb:.4f}")
print(f"Accuracy en test: {test_accuracy_xgb:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_test)}")

Modelo: XGBoost
Accuracy en entrenamiento: 0.9759
Accuracy en test: 0.8571

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      0.70      0.82        10
         1.0       0.79      1.00      0.88        11

    accuracy                           0.86        21
   macro avg       0.89      0.85      0.85        21
weighted avg       0.89      0.86      0.85        21



## Bootstrap

In [ ]:
X_bootstrap, y_bootstrap = resample(X_train, y_train, replace=True, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=42)
rf.fit(X_bootstrap, y_bootstrap)
# 🔹 Accuracy en entrenamiento
y_pred_train = rf.predict(X_train)
train_accuracy_rf = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test (validación)
y_pred_rf = rf.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Mostrar resultados
print(f"Modelo: Random Forest")
print(f"Accuracy en entrenamiento: {train_accuracy_rf:.4f}")
print(f"Accuracy en test: {test_accuracy_rf:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_rf)}")
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(f"Matriz de Confusión para Random Forest:\n", cm_rf)

Modelo: Random Forest
Accuracy en entrenamiento: 0.8795
Accuracy en test: 0.9048

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      0.80      0.89        10
         1.0       0.85      1.00      0.92        11

    accuracy                           0.90        21
   macro avg       0.92      0.90      0.90        21
weighted avg       0.92      0.90      0.90        21

Matriz de Confusión para Random Forest:
 [[ 8  2]
 [ 0 11]]


In [ ]:
# SVM (escalamos los datos bootstrap)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_bootstrap_scaled = scaler.fit_transform(X_bootstrap)

svm = SVC(C=1, kernel='rbf', gamma="scale", probability=True, random_state=42)
svm.fit(X_bootstrap_scaled, y_bootstrap)
# 🔹 Accuracy en entrenamiento
y_pred_train = svm.predict(X_train_scaled)
train_accuracy_svm = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test
y_pred_test = svm.predict(X_test_scaled)
test_accuracy_svm = accuracy_score(y_test, y_pred_test)

# Mostrar resultados
print(f"Modelo: SVM")
print(f"Accuracy en entrenamiento: {train_accuracy_svm:.4f}")
print(f"Accuracy en test: {test_accuracy_svm:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_test)}")

Modelo: SVM
Accuracy en entrenamiento: 0.7470
Accuracy en test: 0.7143

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       0.83      0.50      0.62        10
         1.0       0.67      0.91      0.77        11

    accuracy                           0.71        21
   macro avg       0.75      0.70      0.70        21
weighted avg       0.75      0.71      0.70        21



In [ ]:
xgb = XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.1, random_state=42)
xgb.fit(X_bootstrap, y_bootstrap)

# 🔹 Accuracy en entrenamiento
y_pred_train = xgb.predict(X_train)
train_accuracy_xgb = accuracy_score(y_train, y_pred_train)

# 🔹 Accuracy en test
y_pred_test = xgb.predict(X_test)
test_accuracy_xgb = accuracy_score(y_test, y_pred_test)

# Mostrar resultados
print(f"Modelo: XGBoost")
print(f"Accuracy en entrenamiento: {train_accuracy_xgb:.4f}")
print(f"Accuracy en test: {test_accuracy_xgb:.4f}")
print(f"\nClassification Report (Test):\n{classification_report(y_test, y_pred_test)}")

Modelo: XGBoost
Accuracy en entrenamiento: 0.8554
Accuracy en test: 1.0000

Classification Report (Test):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        11

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21



#Cross Validation para asegurar



##Random Forest

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(model, X_bootstrap, y_bootstrap, cv=10, scoring='accuracy')
print("Accuracy en validación cruzada:", scores)
print("Media:", scores.mean())


Accuracy en validación cruzada: [1.         0.88888889 0.88888889 1.         1.         1.
 1.         1.         0.875      0.875     ]
Media: 0.9527777777777778


##XGBoost

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.1, random_state=42)
scores = cross_val_score(model, X_bootstrap, y_bootstrap, cv=5, scoring='accuracy')
print("Accuracy en validación cruzada:", scores)
print("Media de accuracy en validación cruzada:", scores.mean())

Accuracy en validación cruzada: [1.         0.76470588 0.88235294 1.         0.875     ]
Media de accuracy en validación cruzada: 0.9044117647058822


#Guardar modelo

In [ ]:
import joblib
model = XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_bootstrap, y_bootstrap)
joblib.dump(model, 'modelo_xgboost_cv.joblib')
print("✅ Modelo guardado como modelo.joblib")

✅ Modelo guardado como modelo_xgboost_cv.joblib
